In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
import pickle as pk
import csv
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import re
import time
import warnings
warnings.filterwarnings('ignore')

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train_X_tr = pd.read_csv('D:/transaction_risk_competition/transaction_train_new.csv')
test_X_tr = pd.read_csv('D:/transaction_risk_competition/transaction_round1_new.csv')
label_train = pd.read_csv('D:/transaction_risk_competition/tag_train_new.csv')
label_test = pd.read_csv('D:/transaction_risk_competition/sumbit_uid.csv')

In [4]:
print("len(train_X_tr)",len(train_X_tr))
print("len(test_X_tr)",len(test_X_tr))
print("len(label_train)",len(label_train))
print("len(label_test)",len(label_test))

len(train_X_tr) 264654
len(test_X_tr) 168981
len(label_train) 31179
len(label_test) 31198


In [6]:
train_test_tr=pd.concat([train_X_tr,test_X_tr])
print("len(train_test_tr)",len(train_test_tr))

len(train_test_tr) 433635


In [7]:
del train_X_tr
del test_X_tr

In [8]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [9]:
train_test_tr=train_test_tr[['UID','day','time','trans_amt','bal','acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','channel','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','ip1_sub','geo_code']]

In [11]:
#选取做因子化
##amt_src1 交易类型，余额，花呗做因子化
##trans_type2	线上线下 ，一定要因子化，只有4中，加上none,5中
##market_type	，营销类型，只有2种，加上空值，三种，一定要因子化
##channel 平台，渠道，5种，加上空值6种，适合因子化
values={'market_type':0,'channel':0,'trans_type2':0,'trans_type1':'0','amt_src1':'0','geo_code':'0'} 
train_test_tr.fillna(value=values,inplace = True)


,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub,geo_code
0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,1.0,abc6bc660561e0eb,NaN,NaN,102,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7,wsbq
1,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,105.0,c2f2023d279665b2,NaN,0.0,7a87b1aa20972d48,NaN,NaN,140,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b,wte7
2,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,0.0,cfd035fc7e0a53b2,NaN,NaN,140,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f,ws7n
3,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,0.0,c2f2023d279665b2,NaN,0.0,c71c876b8979028e,NaN,NaN,140,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f,wqj6
4,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,0.0,6628f5ca013e3582,NaN,NaN,140,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9,0


In [12]:
train_test_tr.sort_values(by=['UID','day','time'],inplace=True)


,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub,geo_code
90698,10000,26,12:23:56,5536,100,NaN,NaN,NaN,f29829bc82459191,9a8ee16bde15e38a,105.0,c2f2023d279665b2,NaN,0.0,88aa547576f43f85,NaN,NaN,140,NaN,NaN,deb68f7028682008,aca4977fbe8741e1,IPHONE 5,NaN,34c715882de1a508,1e3ea9498c461cbf,0
134052,10000,26,12:24:17,5536,100,NaN,NaN,NaN,f29829bc82459191,9a8ee16bde15e38a,105.0,c2f2023d279665b2,NaN,0.0,88aa547576f43f85,NaN,NaN,140,NaN,NaN,deb68f7028682008,aca4977fbe8741e1,IPHONE 5,NaN,34c715882de1a508,1e3ea9498c461cbf,0
63298,10001,2,08:31:13,9287,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,105.0,c2f2023d279665b2,NaN,0.0,f3237791ad270d6b,NaN,NaN,140,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,2c1958d001a993d9,bd5e6a16e61e61f7,wsgb
98298,10001,2,08:31:13,154,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,0.0,f3237791ad270d6b,NaN,NaN,140,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,2c1958d001a993d9,bd5e6a16e61e61f7,wsgb
42602,10001,3,07:18:17,15186,61204,65407f8f309309a6,NaN,NaN,155c9e1c32bd0fa2,9fefed0a981dcb7a,102.0,6d55c54c8b1056fb,NaN,0.0,a18cb3d84b9ec04c,NaN,NaN,102,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,bca1b486effef686,8e6d45472f232ed4,wssu


In [14]:
train_test_tr['market_type'], uniques =pd.factorize(train_test_tr['market_type'],sort=True)
del uniques
train_test_tr['trans_type1'], uniques =pd.factorize(train_test_tr['trans_type1'],sort=True)
del uniques
a=train_test_tr['amt_src1'].value_counts().index.tolist()
b=[0,1,2,3,4,5,6,7,8,9,10]+([-1]*(len(a)-11))
dicts=dict(zip(a,b))
train_test_tr['amt_src1']=train_test_tr['amt_src1'].map(dicts)
del a,b,dicts


,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub,geo_code
90698,10000,26,12:23:56,5536,100,NaN,NaN,NaN,3,9a8ee16bde15e38a,105.0,9,NaN,0,88aa547576f43f85,NaN,NaN,140,NaN,NaN,deb68f7028682008,aca4977fbe8741e1,IPHONE 5,NaN,34c715882de1a508,1e3ea9498c461cbf,0
134052,10000,26,12:24:17,5536,100,NaN,NaN,NaN,3,9a8ee16bde15e38a,105.0,9,NaN,0,88aa547576f43f85,NaN,NaN,140,NaN,NaN,deb68f7028682008,aca4977fbe8741e1,IPHONE 5,NaN,34c715882de1a508,1e3ea9498c461cbf,0
63298,10001,2,08:31:13,9287,100,NaN,NaN,NaN,1,NaN,105.0,9,NaN,0,f3237791ad270d6b,NaN,NaN,140,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,2c1958d001a993d9,bd5e6a16e61e61f7,wsgb
98298,10001,2,08:31:13,154,100,NaN,NaN,NaN,0,NaN,105.0,9,NaN,0,f3237791ad270d6b,NaN,NaN,140,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,2c1958d001a993d9,bd5e6a16e61e61f7,wsgb
42602,10001,3,07:18:17,15186,61204,65407f8f309309a6,NaN,NaN,2,9fefed0a981dcb7a,102.0,4,NaN,0,a18cb3d84b9ec04c,NaN,NaN,102,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,bca1b486effef686,8e6d45472f232ed4,wssu


In [15]:
train_test_tr=train_test_tr[['UID','day','time','trans_amt','bal','acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','channel','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','ip1_sub','geo_code']]
train_test_tr.head(5)

,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub,geo_code
90698,10000,26,12:23:56,5536,100,NaN,NaN,NaN,3,9a8ee16bde15e38a,105.0,9,NaN,0,88aa547576f43f85,140,NaN,NaN,deb68f7028682008,aca4977fbe8741e1,IPHONE 5,NaN,34c715882de1a508,1e3ea9498c461cbf,0
134052,10000,26,12:24:17,5536,100,NaN,NaN,NaN,3,9a8ee16bde15e38a,105.0,9,NaN,0,88aa547576f43f85,140,NaN,NaN,deb68f7028682008,aca4977fbe8741e1,IPHONE 5,NaN,34c715882de1a508,1e3ea9498c461cbf,0
63298,10001,2,08:31:13,9287,100,NaN,NaN,NaN,1,NaN,105.0,9,NaN,0,f3237791ad270d6b,140,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,2c1958d001a993d9,bd5e6a16e61e61f7,wsgb
98298,10001,2,08:31:13,154,100,NaN,NaN,NaN,0,NaN,105.0,9,NaN,0,f3237791ad270d6b,140,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,2c1958d001a993d9,bd5e6a16e61e61f7,wsgb
42602,10001,3,07:18:17,15186,61204,65407f8f309309a6,NaN,NaN,2,9fefed0a981dcb7a,102.0,4,NaN,0,a18cb3d84b9ec04c,102,e56819f72c9b7860,1aa9250077f98fea,NaN,09baf2f39bc3dc86,HUAWEI GRA-CL00,8e98757bf596ee35,bca1b486effef686,8e6d45472f232ed4,wssu


In [17]:
train_test_tr_uid1_count=train_test_tr[['acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','channel','device_code1','device_code2','device1','device2','mac1','ip1','ip1_sub','geo_code']].groupby(train_test_tr['UID']).count().add_suffix('_count').reset_index()
train_test_tr_uid1_count.shape

(60643, 20)

# 金额特征

In [18]:
train_test_tr_uid2_channel_amt=train_test_tr.groupby(['UID','channel'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_channel_').unstack()
train_test_tr_uid2_channel_amt.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_channel_amt.columns.ravel()]
train_test_tr_uid2_channel_amt.reset_index(inplace=True)
train_test_tr_uid2_channel_amt.fillna(0.0,inplace=True)
train_test_tr_uid2_channel_amt.shape

(60643, 16)

In [20]:
train_test_tr_uid2_market_type_amt=train_test_tr.groupby(['UID','market_type'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_market_type_').unstack()
train_test_tr_uid2_market_type_amt.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_market_type_amt.columns.ravel()]
train_test_tr_uid2_market_type_amt.reset_index(inplace=True)
train_test_tr_uid2_market_type_amt.fillna(0.0,inplace=True)
train_test_tr_uid2_market_type_amt.shape

(60643, 10)

In [21]:
train_test_tr_uid2_trans_type2_amt=train_test_tr.groupby(['UID','trans_type2'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_trans_type2_').unstack()
train_test_tr_uid2_trans_type2_amt.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_trans_type2_amt.columns.ravel()]
train_test_tr_uid2_trans_type2_amt.reset_index(inplace=True)
train_test_tr_uid2_trans_type2_amt.fillna(0.0,inplace=True)
train_test_tr_uid2_trans_type2_amt.shape

(60643, 16)

In [23]:
train_test_tr_uid2_trans_type1_amt=train_test_tr.groupby(['UID','trans_type1'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_trans_type1_').unstack()
train_test_tr_uid2_trans_type1_amt.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_trans_type1_amt.columns.ravel()]
train_test_tr_uid2_trans_type1_amt.reset_index(inplace=True)
train_test_tr_uid2_trans_type1_amt.fillna(0.0,inplace=True)
train_test_tr_uid2_trans_type1_amt.shape

(60643, 49)

In [24]:
train_test_tr_uid2_amt_src1_amt=train_test_tr.groupby(['UID','amt_src1'])['trans_amt'].agg({'mean','max','min'}).add_prefix('amt_amt_src1_').unstack()
train_test_tr_uid2_amt_src1_amt.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_amt_src1_amt.columns.ravel()]
train_test_tr_uid2_amt_src1_amt.reset_index(inplace=True)
train_test_tr_uid2_amt_src1_amt.fillna(0.0,inplace=True)
train_test_tr_uid2_amt_src1_amt.shape

(60643, 37)

# 余额特征

In [25]:
train_test_tr_uid2_channel_bal=train_test_tr.groupby(['UID','channel'])['bal'].agg({'mean','max','min'}).add_prefix('amt_channel_').unstack()
train_test_tr_uid2_channel_bal.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_channel_bal.columns.ravel()]
train_test_tr_uid2_channel_bal.reset_index(inplace=True)
train_test_tr_uid2_channel_bal.fillna(0.0,inplace=True)
train_test_tr_uid2_channel_bal.shape

(60643, 16)

In [27]:
train_test_tr_uid2_market_type_bal=train_test_tr.groupby(['UID','market_type'])['bal'].agg({'mean','max','min'}).add_prefix('amt_market_type_').unstack()
train_test_tr_uid2_market_type_bal.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_market_type_bal.columns.ravel()]
train_test_tr_uid2_market_type_bal.reset_index(inplace=True)
train_test_tr_uid2_market_type_bal.fillna(0.0,inplace=True)
train_test_tr_uid2_market_type_bal.shape

(60643, 10)

In [28]:
train_test_tr_uid2_trans_type2_bal=train_test_tr.groupby(['UID','trans_type2'])['bal'].agg({'mean','max','min'}).add_prefix('amt_trans_type2_').unstack()
train_test_tr_uid2_trans_type2_bal.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_trans_type2_bal.columns.ravel()]
train_test_tr_uid2_trans_type2_bal.reset_index(inplace=True)
train_test_tr_uid2_trans_type2_bal.fillna(0.0,inplace=True)
train_test_tr_uid2_trans_type2_bal.shape

(60643, 16)

In [29]:
train_test_tr_uid2_trans_type1_bal=train_test_tr.groupby(['UID','trans_type1'])['bal'].agg({'mean','max','min'}).add_prefix('amt_trans_type1_').unstack()
train_test_tr_uid2_trans_type1_bal.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_trans_type1_bal.columns.ravel()]
train_test_tr_uid2_trans_type1_bal.reset_index(inplace=True)
train_test_tr_uid2_trans_type1_bal.fillna(0.0,inplace=True)
train_test_tr_uid2_trans_type1_bal.shape

(60643, 49)

In [30]:
train_test_tr_uid2_amt_src1_bal=train_test_tr.groupby(['UID','amt_src1'])['bal'].agg({'mean','max','min'}).add_prefix('amt_amt_src1_').unstack()
train_test_tr_uid2_amt_src1_bal.columns = [x[0]+"_"+str(x[1]) for x in train_test_tr_uid2_amt_src1_bal.columns.ravel()]
train_test_tr_uid2_amt_src1_bal.reset_index(inplace=True)
train_test_tr_uid2_amt_src1_bal.fillna(0.0,inplace=True)
train_test_tr_uid2_amt_src1_bal.shape

(60643, 37)

# 日期，平均每日交易次数

In [32]:
train_test_tr_uid_day = train_test_tr[['UID', 'day']]
train_test_tr_uid_day.drop_duplicates(['UID', 'day'], 'first', inplace=True)
train_test_tr_uid1_day_count = train_test_tr_uid_day[['day']].groupby(train_test_tr_uid_day['UID']).count().add_suffix('_count').reset_index()
train_test_tr_uid1_day_count.shape

(60643, 2)

In [33]:
train_test_tr['day_time'] = train_test_tr.apply(lambda x: str(x['day']) + "_" + str(x['time']) , axis=1)
train_test_tr_uid1_day_time_count = train_test_tr[['day_time']].groupby(train_test_tr['UID']).count().add_suffix('_count').reset_index()
train_test_tr_uid1_day_time_count.shape

(60643, 2)

In [34]:
train_test_tr_uid1_count_day_avg = pd.merge(train_test_tr_uid1_day_time_count,train_test_tr_uid1_day_count,how='left',on='UID')
train_test_tr_uid1_count_day_avg['count_day_avg']=train_test_tr_uid1_count_day_avg['day_time_count']/train_test_tr_uid1_count_day_avg['day_count']
del train_test_tr_uid1_day_count,train_test_tr_uid1_day_time_count
train_test_tr_uid1_count_day_avg.shape

(60643, 4)

# 相邻两次交易时间差

In [43]:
#需要先排序
train_test_tr.sort_values(by=['UID','day','time'],inplace=True)
train_test_tr['day_shift'] = train_test_tr.groupby('UID')['day'].shift(-1)
train_test_tr_uid_day_diff_avg_max_min=train_test_tr.groupby(['UID'])['day_shift'].agg({'mean','max','min'}).add_prefix('day_shift_').reset_index()
train_test_tr_uid_day_diff_avg_max_min.fillna(0.0,inplace=True)
train_test_tr_uid_day_diff_avg_max_min.shape

(60643, 4)

# 特征merge left

In [ ]:
feature_tr_train_test = pd.merge(train_test_tr_uid1_count_day_avg, train_test_tr_uid_day_diff_avg_max_min, how='left', on='UID')

feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_amt_src1_bal, how='left', on='UID')
feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_trans_type1_bal, how='left', on='UID')
feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_trans_type1_bal, how='left', on='UID')
feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_market_type_bal, how='left', on='UID')
feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_channel_bal, how='left', on='UID')

feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_amt_src1_amt, how='left', on='UID')
feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_trans_type1_amt, how='left', on='UID')
feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_trans_type1_amt, how='left', on='UID')
feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_market_type_amt, how='left', on='UID')
feature_tr_train_test = pd.merge(feature_tr_train_test, train_test_tr_uid2_channel_amt, how='left', on='UID')

feature_tr_train_test.shape

(60643, 319)

In [ ]:
 # 分别和训练集的label 和提交的uid 关联，分开
 
 
 